In [6]:
from IPython.display import Javascript
timeout = 185 # minutes
Javascript("""let timerId = setInterval(() => IPython.notebook.kernel.execute("kernel_survivor=1"), 300000);
setTimeout(() => { clearInterval(timerId); alert('fin de cession'); },"""+ str(timeout*1000*60)+")")

<IPython.core.display.Javascript object>

In [7]:
Javascript("""Jupyter.actions.call('jupyter-notebook:hide-header')""")

<IPython.core.display.Javascript object>

In [8]:
Javascript("""Jupyter.actions.call('jupyter-notebook:hide-toolbar')""")

<IPython.core.display.Javascript object>

In [17]:
# Parameters:


In [12]:
from ipywidgets import FileUpload, Button, HBox, Layout, Dropdown, VBox,Label, Box, Text, Password, Accordion, Image
from ipywidgets import HTML as html
from IPython.core.display import HTML
from IPython.display import Javascript
import os
import time

#print(os.environ['JUPYTERHUB_API_TOKEN'])
try:
    print(URL)
except:
    URL = None
    
site_file = open("site.txt","r")
site_data = site_file.read().strip()
site_file.close()
print(site_data)


    
lock_bou3 = False
new = 0

message = Label(
    value = "")

bou = Button(
    description='New Notebook',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='New Notebook',
    icon='' # (FontAwesome names without the `fa-` prefix)
)

bou5 = Button(
    description='New .py',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='New .py',
    icon='' # (FontAwesome names without the `fa-` prefix)
)



f = FileUpload(
    description = "Upload notebook",
    accept='.ipynb',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)


bou2 = Button(
    description='Terminal',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Terminal',
    icon='' # (FontAwesome names without the `fa-` prefix)
)

site = Text(
    description = "site",
    disabled = False,
    tooltip = "site",
    value = site_data)

login = Text(
    description = "login",
    disabled = False,
    tooltip = 'login')

password = Password(
    description = "password",
    disabled = False,
    tooltip = 'password')

bou_log = Button(
    description='se connecter',
    disabled = True,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='se connecter',
    icon='' # (FontAwesome names without the `fa-` prefix)
)

download_bou = Button(
    description='Save all',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Save all',
    icon='download' # (FontAwesome names without the `fa-` prefix)
)


upload_bou = FileUpload(
    description='Upload zip',
    disabled=False,
    accept=".zip",
    multiple=False,
    tooltip='Upload zip',
    icon='upload' # (FontAwesome names without the `fa-` prefix)

)


lien = html(
    value = "")

def zipToServer(b):
    lien.value = ""
    path = os.getcwd()
    os.chdir("/home/jovyan/")
    message.value = "decompression du fichier de sauvegarde"
    v = upload_bou.value
    
    with open(next(iter(upload_bou.value)), "w+b") as i:
        i.write(upload_bou.data[0])
    a = os.popen("""unzip -o {}""".format(next(iter(upload_bou.value))))
    m = ""
    for ligne in a:
        m += str(ligne)
    j = open("log_cadaver.txt","a")
    j.write("\n\n")
    j.write(m)
    j.close()
    os.chdir(path)
    message.value = "opération terminée avec succès."
    lien.value = "Cliquez sur le lien pour <a href=../tree/Perso target = _blank>accéder à votre répertoire</a>"
    change2()
    

def reset(b):
    lien.value = ""
    message.value = ""
    
def saveToAtrium(b):
    lien.value = ""
    Javascript("""jupyter.actions.call('jupyter-notebook:save-notebook');""")
    path = os.getcwd()
    os.chdir("/home/jovyan/")
    t = time.localtime()
    message.value = "création du fichier de sauvegarde"
    file = "nsipremiere{:4d}{:02d}{:02d}{:02d}{:02d}{:02d}.zip".format(t.tm_year,t.tm_mon,t.tm_mday,t.tm_hour,t.tm_min,t.tm_sec)
    a = os.popen("""zip -r {} Perso""".format(file))
    m = ""
    for ligne in a:
        m += str(ligne)
    j = open("log_cadaver.txt","a")
    j.write("\n\n")
    j.write(m)
    j.close()
    
    j = open(".netrc","r")
    data = j.readlines()
    j.close()
    data = data[1]
    login = data.split(" ")[1].strip()
    
    message.value = "upload du fichier de sauvegarde vers atrium"
    j = open("datadav","w")
    j.write("cd {}\ncd document_library\nput /home/jovyan/nsipremiere{:4d}{:02d}{:02d}{:02d}{:02d}{:02d}.zip\nexit".format(login,t.tm_year,t.tm_mon,t.tm_mday,t.tm_hour,t.tm_min,t.tm_sec))
    j.close()
    a = os.popen("""cadaver -r datadav https://""" + site_data + """/webdav""")
    m = ""
    for ligne in a:
        m += str(ligne)
    j = open("log_cadaver.txt","a")
    j.write("\n\n")
    j.write(m)
    j.close()
    if "succeeded" in m:
        message.value = "operation terminée avec succès"
    else:
        message.value = m
    message.value = """Sauvegarde réalisée avec succès sur le serveur."""
    lien.value = "Cliquez sur le lien pour <a href={} download>télécharger la sauvegarde</a>".format(file)
    
    os.chdir(path)
    
left_column = VBox([html("""<h4 style="text-align:center; width:100%;">Log Atrium</h4>"""),login,password,bou_log,message],
                   layout=Layout(min_width="350px",flex_flow="column wrap", align_items='center'))
middle_column = VBox([html("""<h4 style="text-align:center; width:100%;">Sauvegarder/Charger cours complet</h4>"""),download_bou,upload_bou,lien],
                   layout=Layout(min_width="350px",flex_flow="column wrap", align_items='center'))

def change(b):
#    (os.path.join(root, i),
    global lock_bou3,base
    if bou3.value !="---":
        display(Javascript('window.open("../notebooks/'+base[bou3.value]+bou3.value+'");'))
    #else:
    #    lock_bou3 = False

def change2():
    global liens,images,titres,accordion
    liens,images,titres = changeLinks()
    k = list(liens.keys())
    k.sort()
    children = []
    for cle in k:
        if images[cle] is not None:
            children.append(HBox([images[cle],html(liens[cle])]))
        else:
            children.append(HBox([html(liens[cle])]))
    accordion.children = children
    i = 0
    for cle in k:
        accordion.set_title(i,titres[cle])
        i += 1
    if i>0:
        accordion.selected_index = 0

def on_change(b):
    l = login.value
    p = password.value
    s = site.value
    if p!="" and l!="" and s!="":
        bou_log.disabled = False
    else:
        bou_log.disabled = True
        
    
def on_use(b):
    v = f.value
    if len(v)>6 and v in os.listdir():
        v = v[:-6]+(1)+v[-6:]
    with open(next(iter(f.value)), "w+b") as i:
        i.write(f.data[0])
    os.popen("jupyter trust '{}'".format(next(iter(f.value))))
    url = "../notebooks/"+ next(iter(f.value))
    display(Javascript('window.open("{url}");'.format(url=url)))
    #display(HTML('<a href="../notebooks/note.ipynb">go !</a>'))

def on_click2(b):
    display(Javascript('window.open("../terminals/1");'))
    #display(HTML('<a href="../terminals/1">go !</a>'))
    
def on_click(b):
    global new
    new += 1
    i = open("new.ipynb","r")
    j = open("/home/jovyan/Perso/new{}.ipynb".format(new),"w")
    j.write(i.read())
    i.close()
    j.close()
    url = "../notebooks/Perso/new{}.ipynb".format(new)
    display(Javascript('window.open("{url}");'.format(url=url)))
    # display(HTML('<a href="../notebooks/new.ipynb">go !</a>'))
    
def on_click3(b):
    global new
    new += 1
    j = open("/home/jovyan/Perso/new{}.py".format(new),"w")
    j.close()
    url = "../edit/Perso/new{}.py".format(new)
    display(Javascript('window.open("{url}");'.format(url=url)))
    # display(HTML('<a href="../notebooks/new.ipynb">go !</a>'))
     
def on_click_log(b):
    site.disabled = True
    login.disabled = True
    password.disabled = True
    bou_log.on_click(on_click_log,remove=True)
    bou_log.disabled = True
    bou_log.description = "changer de compte"
    bou_log.tooltip = "changer de compte"
    message.value = "Try to connect, please wait..."
    
    j = open(".netrc","w")
    j.write("machine "+site.value.strip()+"\n")
    j.write("login "+login.value.strip()+"\n")
    j.write("password "+password.value.strip())
    j.close()
    
    j = open("datadav","w")
    j.write("cd {}\ncd document_library\nls\nexit".format(login.value.strip()))
    j.close()
    a = os.popen("""cadaver -r datadav https://www.atrium-sud.fr/webdav""")
    m = ""
    for ligne in a:
        m += str(ligne)
    j = open("log_cadaver.txt","a")
    j.write("\n\n")
    j.write(m)
    j.close()
    if "Connection to" in m:
        message.value = "Connected as {}. Downloading data...".format(login.value.strip())
    else:
        message.value = "Connection failed. Check your login and password."
        bou_log.disabled = False
        bou_log.on_click(change_compte)
        return
    
    # trouver la dernière sauvegarde
    m = m.split("\n")
    liste = []
    for ligne in m:
        ligne = ligne.strip()
        ligne = ligne.split(" ")
        if ligne[0] != "Coll:" and "nsipremiere" in ligne[0]:
            liste.append(ligne[0].strip())
           
    if len(liste) > 0:
        liste.sort() 
        j = open("datadav","w")
        j.write("cd {}\ncd document_library\nget {}".format(login.value.strip(),liste[-1]))
        while len(liste) > 10:
                j.write("\ndelete {}".format(liste.pop(0)))
        j.write("\nexit")
        j.close()
        a = os.popen("""cadaver -r datadav https://www.atrium-sud.fr/webdav""")
        m = ""
        for ligne in a:
            m += str(ligne)
        j = open("log_cadaver.txt","a")
        j.write("\n\n")
        j.write(m)
        j.close()
        a = os.popen("""unzip -o {}""".format(liste[-1]))
        m = ""
        for ligne in a:
            m += str(ligne)
        j = open("log_cadaver.txt","a")
        j.write("\n\n")
        j.write(m)
        j.close()
    else:
        os.popen("""mkdir /home/jovyan/Perso""")
    
    # trouver le cours
    j = open("datadav","w")
    j.write("cd nsipremiere-9012620\ncd document_library\nls\nexit")
    j.close()
    a = os.popen("""cadaver -r datadav https://www.atrium-sud.fr/webdav""")
    m = ""
    for ligne in a:
        m += str(ligne)
    j = open("log_cadaver.txt","a")
    j.write("\n\n")
    j.write(m)
    j.close()
    
    m = m.split("\n")
    liste = []
    for ligne in m:
        ligne = ligne.strip()
        ligne = ligne.split(" ")
        if ligne[0] != "Coll:" and "nsipremiere" in ligne[0]:
            liste.append(ligne[0].strip())
    liste.sort()        
    j = open("datadav","w")
    j.write("cd nsipremiere-9012620\ncd document_library\nget {}\nexit".format(liste[-1]))
    j.close()
    a = os.popen("""cadaver -r datadav https://www.atrium-sud.fr/webdav""")
    m = ""
    for ligne in a:
        m += str(ligne)
    j = open("log_cadaver.txt","a")
    j.write("\n\n")
    j.write(m)
    j.close()
    a = os.popen("""unzip -o {}""".format(liste[-1]))
    m = ""
    for ligne in a:
        m += str(ligne)
    j = open("log_cadaver.txt","a")
    j.write("\n\n")
    j.write(m)
    j.close()
    message.value = "Connected as {}".format(login.value.strip())
    change2()
    
    

def change_compte(b):
    login.value = ""
    password.value = ""
    message.value = ""
    site.disabled = False
    login.disabled = False
    password.disabled = False
    bou_log.disabled = True
    
    j = open(".netrc","w")
    j.write("machine "+site.value.strip()+"\n")
    j.close()

    bou_log.on_click(change_compte,remove=True)
    bou_log.description = "se connecter"
    bou_log.tooltip = "se connecter"
    bou_log.on_click(on_click_log)

download_bou.on_click(saveToAtrium)
upload_bou.observe(zipToServer, "data")
bou.on_click(on_click)
bou2.on_click(on_click2)
bou5.on_click(on_click3)
bou_log.on_click(on_click_log)
login.observe(on_change,"value")
password.observe(on_change,"value")
site.observe(on_change,"value")
f.observe(on_use,"data")

display(HTML("""<h1 style="text-align:center; width:100%;"> Cours de Premiere NSI 2021/2022 </h1>"""))
display(HTML("""<h3 style="text-align:center; width:100%;"> Environnement JUPYTER </h3>"""))
display(Box([left_column,middle_column,VBox([html("""<h4 style="text-align:center; width:100%;">Créer/Charger un notebook</h4>"""),bou,bou5,f],layout=Layout(min_width="200px",flex_flow="column wrap", align_items='center')),VBox([html("""<h4 style="text-align:center; width:100%;">Terminal Linux</h4>"""),bou2],layout=Layout(flex_flow="column wrap",min_width="200px", align_items='center'))],layout=Layout(width="100%",height="auto",display="flex",flex_flow="row wrap",min_width='800px',justify_content='space-around')))
display(HTML("""<h3 style="text-align:center; width:100%;"> Sommaire </h3>"""))

def changeLinks():
    liens = {}
    images = {}
    titres = {}
    for root1, dirs1, files1 in os.walk(os.getcwd()): 
        for d in dirs1:
            for root, dirs, files in os.walk(os.path.join(root1,d)):
                if root[:27] == "/home/jovyan/Cours/Chapitre" and root[-2:].isnumeric():
                    n = root[-2:]
                    if n not in liens.keys():
                        liens[n] = {}
                        images[n] = None
                        titres[n] = ""
                    for i in files:
                        g = os.path.join(root.replace("Cours","Perso"),i)
                        if not os.path.exists(root.replace("Cours","Perso")):
                            os.popen("""mkdir {}""".format(root.replace("Cours","Perso")))
                        f = os.path.join(root, i)
                        
                        if f[-6:] == ".ipynb":
                            if not os.path.exists(g):
                                os.popen("""cp {} {}""".format(f,g))
                            num = i.split("_")[0]
                            #print(num)
                            #print(i)
                            
                            num2 = num.split(".")
                            if "cor" in num2:
                                l = "<ul>"*len(num2)+"<li>"+"""<a href="../notebooks/{}" target="_blank" style="color:#FF0000";>{}</a>""".format(g[13:],i[len(num)+1:-6])+"</li>"+"</ul>"*len(num2)+'</p>'
                            elif "app" in num2:
                                l = "<ul>"*len(num2)+"<li>"+"""<a href="../apps/{}" target="_blank" style="color:#00CC00";>{}</a>""".format(g[13:],i[len(num)+1:-6])+"</li>"+"</ul>"*len(num2)
                            else:
                                l = "<ul>"*len(num2)+"<li>"+"""<a href="../notebooks/{}" target="_blank">{}</a>""".format(g[13:],i[len(num)+1:-6])+"</li>"+"</ul>"*len(num2)
                            if num[-3:] == "cor":
                                num = num[:-3]+"0"
                            elif num[-3:] == "app":
                                num = num[:-3]+"00"
                            liens[n][num] = l
                        elif f[-9:] == "image.txt":
                            file = open(f)
                            data = file.readline()
                            file.close()
                            images[n] = html(value = '<img src="{}" width="400px">'.format(data))
                        elif f[-4:] == ".png" or f[-4:] == ".jpg":
                            images[n] = Image(
                                        value = open(f, "rb").read(),
                                        format = f[-3:],
                                        width = 400,
                                        height = 400,
                                        )
                        elif f[-9:] == "titre.txt":
                            file = open(f)
                            data = file.readline()
                            file.close()
                            titres[n] = data
                elif root[:27] == "/home/jovyan/Perso/Chapitre" and root[-2:].isnumeric():
                    n = root[-2:]
                    if n not in liens.keys():
                        liens[n] = {}
                        images[n] = None
                        titres[n] = ""
                    for i in files: 
                        f = os.path.join(root, i)
                        if f[-6:] == ".ipynb":
                            num = i.split("_")[0]
                            #print(num)
                            #print(i)
                            
                            num2 = num.split(".")
                            if "cor" in num2:
                                l = "<ul>"*len(num2)+"<li>"+"""<a href="../notebooks/{}" target="_blank" style="color:#FF0000";>{}</a>""".format(f[13:],i[len(num)+1:-6])+"</li>"+"</ul>"*len(num2)+'</p>'
                            elif "app" in num2:
                                l = "<ul>"*len(num2)+"<li>"+"""<a href="../apps/{}" target="_blank" style="color:#00CC00";>{}</a>""".format(f[13:],i[len(num)+1:-6])+"</li>"+"</ul>"*len(num2)
                            else:
                                l = "<ul>"*len(num2)+"<li>"+"""<a href="../notebooks/{}" target="_blank">{}</a>""".format(f[13:],i[len(num)+1:-6])+"</li>"+"</ul>"*len(num2)
                            if num[-3:] == "cor":
                                num = num[:-3]+"0"
                            elif num[-3:] == "app":
                                num = num[:-3]+"00"
                            liens[n][num] = l
        k = list(liens.keys())
        k.sort()
        liens2 = {}
        for cle in k:
            k2 = list(liens[cle].keys())
            k2.sort()
            l = ""
            for cle2 in k2:
                l += liens[cle][cle2]
            liens2[cle] = l
        return liens2,images,titres
    
liens,images,titres = changeLinks()
#print(liens,images,titres)
k = list(liens.keys())
#print("\n")
#print(k)
k.sort()
#print("\n")
#print([images[cle] for cle in k])
children = []
for cle in k:
    if images[cle] is not None:
        children.append(HBox([images[cle],html(liens[cle])]))
    else:
        children.append(HBox([html(liens[cle])]))


accordion = Accordion(children=children)
i = 0
for cle in k:
    accordion.set_title(i,titres[cle])
    i += 1
    





        
    



display(accordion)

fce1a4f8a431481189b762b93f919993
KeysView(environ({'GCS_PROXY_PORT': 'tcp://10.3.244.175:80', 'CONDA_SHLVL': '1', 'PYTHONUNBUFFERED': '1', 'LC_ALL': 'en_US.UTF-8', 'JUPYTERHUB_CLIENT_ID': 'jupyterhub-user-jcamponovo-jupyteratrium-dc7id2sm', 'APP_BASE': '/srv', 'KERNEL_PYTHON_PREFIX': '/srv/conda/envs/notebook', 'CONDA_EXE': '/srv/conda/bin/conda', 'PROXY_API_PORT': 'tcp://10.3.244.201:8001', 'FEDERATION_REDIRECT_PORT': 'tcp://10.3.244.205:80', 'PROD_INGRESS_NGINX_DEFAULTBACKEND_SERVICE_HOST': '10.3.249.179', 'PROD_PROMETHEUS_SERVER_PORT_80_TCP_PROTO': 'tcp', 'PROXY_API_PORT_8001_TCP_PORT': '8001', 'FEDERATION_REDIRECT_PORT_80_TCP_PROTO': 'tcp', 'JUPYTERHUB_ADMIN_ACCESS': '1', 'PROD_KUBE_STATE_METRICS_SERVICE_PORT': '8080', 'PROD_PROMETHEUS_SERVER_SERVICE_HOST': '10.3.241.169', 'LANG': 'en_US.UTF-8', 'GCS_PROXY_PORT_80_TCP_PORT': '80', 'MATOMO_PORT_9000_TCP_PORT': '9000', 'GCS_PROXY_PORT_80_TCP_PROTO': 'tcp', 'PROXY_PATCHES_PORT_80_TCP_PROTO': 'tcp', 'HUB_PORT_8081_TCP_PROTO': 'tcp', 'P

AttributeError: 'list' object has no attribute 'split'